In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ampl_utils import ampl_license
from amplpy import AMPL, ampl_notebook

from models import CPOP, NAOP

from dataloader import Network

from itertools import product

solver='cplex'
ampl_license_uuid = ampl_license()

ampl = ampl_notebook(
    modules=[solver],
    license_uuid=ampl_license_uuid)

Licensed to AMPL Academic Community Edition License for <s207399@student.pg.edu.pl>.


In [9]:
cpop = CPOP(Network('dognet'))
cpop.load_data([[4, 1]], 1)
cpop.solve()
cpop.report()

CPLEX 22.1.2: optimal solution; objective 5
0 simplex iterations


{'s': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 'Y': 5.0}

In [12]:
naop = NAOP(Network('dognet'))
naop.load_data([[2, 4, 8]], 2)
naop.solve()


naop.report()

CPLEX 22.1.2: optimal solution; objective 3
23 simplex iterations


{'a': [0, 0, 1, 0, 1, 0, 0, 0, 0, 0], 'Z': 3.0}